In [ ]:
https://docs.google.com/spreadsheets/d/1uN_4ds9T59BicDncCTXqLxSCDtiOxgktCZ8w1vmEPaU/pub?output=csv

In [53]:
## all imports
from IPython.display import HTML
import chardet
import io
import numpy as np
import os
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import requests
import operator
import socket
import _pickle
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

#from secret import *

### IMPORT CSV FILE

In [54]:
import requests
r = requests.get('https://docs.google.com/spreadsheets/d/1uN_4ds9T59BicDncCTXqLxSCDtiOxgktCZ8w1vmEPaU/pub?output=csv')
data = r.content

### READ CSV FILE

In [55]:
df = pd.read_csv(io.BytesIO(data), infer_datetime_format=True,
                           parse_dates=['Date'],
                           dayfirst=True )

In [56]:
df.head()

,City,Date,Kick Off (local),Home Team,Away Team,Venue,Home Score,Away Score,Home Goals,Home Behinds,Away Goals,Away Behinds,Home Odds,Away Odds,Rain,URL
0,Melbourne,2017-04-16,15:20,Hawthorn,Geelong,MCG,48,134,6,12,20,14,2.87,1.42,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...
1,Brisbane,2017-04-16,16:40,Brisbane,Richmond,Gabba,67,119,10,7,17,17,3.17,1.35,0.0,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...
2,Melbourne,2017-04-16,15:20,Collingwood,St Kilda,Etihad Stadium,55,69,7,13,9,15,1.62,2.29,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...
3,Adelaide,2017-04-15,19:10,Adelaide,Essendon,Adelaide Oval,153,88,24,9,13,10,1.16,5.29,0.0,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...
4,Melbourne,2017-04-15,19:25,Carlton,Gold Coast,Etihad Stadium,82,108,12,10,17,6,2.05,1.77,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...


### ONLY USE DATA WHERE RAIN HAS A VALUE

In [57]:
df = df[df.Rain >= 0.0]

### CONVERT ALL RAIN COLUMN TO FLOAT

In [59]:
df['Rain'].apply(float)

1        0.0
3        0.0
9        0.0
18       0.0
22       0.2
23       0.0
25       2.8
26       2.8
28       0.0
29       0.0
30      34.0
31       0.0
32       0.0
33       2.0
34       0.0
35       0.0
36       0.0
37       0.0
38       0.6
39       0.0
40      15.2
41       0.0
42       2.8
43       0.6
44       8.8
45       7.0
46       0.0
47       0.0
48       1.0
49       0.2
        ... 
1512     0.0
1513     0.0
1515     0.0
1516     0.0
1517     0.8
1519     0.0
1520    13.0
1521     0.0
1522     2.6
1523     0.0
1524     0.0
1525     1.4
1526     0.0
1527     0.0
1528     1.0
1529     0.0
1530     1.4
1531     1.0
1532     6.4
1533     0.0
1534     0.0
1535     0.2
1536    14.8
1537     5.8
1538     0.4
1539     0.0
1540     0.0
1541     8.8
1542     2.6
1543     0.2
Name: Rain, dtype: float64

In [76]:
test = df[df.Rain >= 3.0]
print(test.Rain.count())

230


## users who are female and programmers
selected_users = users[(users.sex == 'F') & 
                       (users.occupation == 'programmer')]

## show statistic summary
print selected_users.describe()

## alternatives:
print selected_users.age.mean()
print selected_users['age'].mean(